# Web Scraping Name, Prices, URLs, Ratings and the total numbers of the reviews

- This notebook contains the scraper of all hotels in NYC from Tripadvaisor webside. 

In [1]:
from selenium import webdriver
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Helper functions

In [2]:
def getting_names(soup_):
        names = soup_.findAll('div', {"class": "listing_title"})
        list_names = [i.text for i in names]
        return list_names

In [3]:
def getting_prices(driver_):
    prizes = driver_.find_elements_by_xpath("//div[@class='price-wrap ']")
    list_prize=[prize.text for prize in prizes]
    return list_prize

In [4]:
def getting_total(soup_):
    number_reviews = soup_.findAll('a', {"class": "review_count"})
    list_total_reviews = [i.text for i in number_reviews]
    return list_total_reviews

In [5]:
def getting_ratings(soup_):
    rating = soup.findAll('a', {"data-clicksource":"BubbleRating"})
    ratings = [element.attrs['alt']  for element in rating]
    return ratings


In [6]:
def getting_urls(soup_):
    domain = "https://www.tripadvisor.com/"
    list_urls = [domain + i.attrs['href'] for i in soup_.findAll('a',class_="review_count")]
    return list_urls

## Scraper

In [7]:
names = []
prices = []
total = []
url = []
ratings = []
for page in range(1,30):
    if page == 1:
        chromedriver = "/Users/cristinamulas/Downloads/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
        driver = webdriver.Chrome(chromedriver)
        driver.get("https://www.tripadvisor.com/Hotels-g60763-New_York_City_New_York-Hotels.html")
        driver.maximize_window()
        driver.implicitly_wait(10)
        nextbutton=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='taplc_main_pagination_bar_dusty_hotels_resp_0']/div/div/div/a")))
        driver.execute_script("arguments[0].click();",nextbutton)
    else:
        time.sleep(10)
        driver.find_element_by_xpath("//a[contains(text(),'Next')]").click()
        if page == 28:
            break
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    names += getting_names(soup)
    try:
         prices += getting_prices(driver)
    except:
            pass

    total += getting_total(soup)
    try:
        url += getting_urls(soup)

    except:
            pass
    ratings += getting_ratings(soup)
       
df = pd.DataFrame([names, prices, total, url, ratings]).transpose()
df.columns = ['hotel_names','prices', 'total_reviews', 'url', 'ratings']  

<IPython.core.display.Javascript object>

In [8]:
len(df)

886

## Save the data into a csv file

In [35]:
df.to_csv('hotel_features.csv', index=False)
